In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-dataset/hour.csv
/kaggle/input/bike-sharing-dataset/Readme.txt
/kaggle/input/bike-sharing-dataset/day.csv
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/hour.csv
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/Readme.txt
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/day.csv


In [2]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv("../input/bike-sharing-dataset/day.csv")

In [4]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [5]:
df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [6]:
df.drop(['instant', 'dteday'], inplace=True ,axis=1)

In [7]:
correlations = df.corrwith(df['cnt'])
print(correlations)

season        0.406100
yr            0.566710
mnth          0.279977
holiday      -0.068348
weekday       0.067443
workingday    0.061156
weathersit   -0.297391
temp          0.627494
atemp         0.631066
hum          -0.100659
windspeed    -0.234545
casual        0.672804
registered    0.945517
cnt           1.000000
dtype: float64


Features with positive correlation are: 
* **season**
* **yr**: year
* **mnth**: mmonths
* **weekday**: days of the week
* **workingday**: whether the day is neither a weekend nor holiday
* **temp**: temperature in Celsius
* **atemp**: "feels like" temperature in Celsius
* **casual**: number of non-registered user rentals initiated
* **registered**: number of registered user rentals initiated

The feature with highest correlation is **'registered'**.

Features with negative correlation are: 
* **holiday**: whether the day is considered a holiday
* **weathersit**
* **hum**: humidity
* **windspeed**

In [8]:
df.drop(['casual', 'registered'], inplace=True ,axis=1)

In [9]:
df.head()


,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600


In [10]:
import category_encoders as ce

In [11]:
encoder = ce.OneHotEncoder(cols=('season','yr','mnth','holiday','weekday','weathersit'), use_cat_names = True)

In [12]:
from sklearn.model_selection import train_test_split


In [13]:
encoded = encoder.fit_transform(df)
new_df = pd.concat([df.drop(['season','yr','mnth','holiday','weekday','weathersit'],axis=1),encoded],axis=1)

In [14]:
new_df.head()

,workingday,temp,atemp,hum,windspeed,cnt,season_1.0,season_2.0,season_3.0,season_4.0,...,weekday_5.0,workingday,weathersit_2.0,weathersit_1.0,weathersit_3.0,temp,atemp,hum,windspeed,cnt
0,0,0.344167,0.363625,0.805833,0.160446,985,1,0,0,0,...,0,0,1,0,0,0.344167,0.363625,0.805833,0.160446,985
1,0,0.363478,0.353739,0.696087,0.248539,801,1,0,0,0,...,0,0,1,0,0,0.363478,0.353739,0.696087,0.248539,801
2,1,0.196364,0.189405,0.437273,0.248309,1349,1,0,0,0,...,0,1,0,1,0,0.196364,0.189405,0.437273,0.248309,1349
3,1,0.200000,0.212122,0.590435,0.160296,1562,1,0,0,0,...,0,1,0,1,0,0.200000,0.212122,0.590435,0.160296,1562
4,1,0.226957,0.229270,0.436957,0.186900,1600,1,0,0,0,...,0,1,0,1,0,0.226957,0.229270,0.436957,0.186900,1600


In [15]:
X = new_df.drop(['cnt','workingday'], axis = 1)
y = new_df['cnt']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
LR = LinearRegression()

In [19]:
LR.fit(X_train, y_train)

LinearRegression()

In [20]:
pred = LR.predict(X_train)

In [21]:
from sklearn.metrics import mean_squared_error,r2_score

In [22]:
mean_squared_error(y_train, pred,squared=False)

758.9883042942054

In [23]:
r2_score(y_train, pred)

0.8416931843702429

In [24]:
X_test

,temp,atemp,hum,windspeed,season_1.0,season_2.0,season_3.0,season_4.0,yr_0.0,yr_1.0,...,weekday_3.0,weekday_4.0,weekday_5.0,weathersit_2.0,weathersit_1.0,weathersit_3.0,temp,atemp,hum,windspeed
703,0.475833,0.469054,0.733750,0.174129,0,0,0,1,0,1,...,0,0,0,0,1,0,0.475833,0.469054,0.733750,0.174129
33,0.186957,0.177878,0.437826,0.277752,1,0,0,0,1,0,...,0,1,0,0,1,0,0.186957,0.177878,0.437826,0.277752
300,0.330833,0.318812,0.585833,0.229479,0,0,0,1,1,0,...,0,0,1,1,0,0,0.330833,0.318812,0.585833,0.229479
456,0.425833,0.417287,0.676250,0.172267,0,1,0,0,0,1,...,0,0,0,1,0,0,0.425833,0.417287,0.676250,0.172267
633,0.550000,0.544179,0.570000,0.236321,0,0,0,1,0,1,...,0,0,0,0,1,0,0.550000,0.544179,0.570000,0.236321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,0.470000,0.457046,0.812917,0.197763,0,0,0,1,1,0,...,0,1,0,1,0,0,0.470000,0.457046,0.812917,0.197763
516,0.680000,0.631329,0.492917,0.195279,0,1,0,0,0,1,...,0,1,0,0,1,0,0.680000,0.631329,0.492917,0.195279
533,0.592500,0.571975,0.598750,0.144904,0,1,0,0,0,1,...,0,0,0,0,1,0,0.592500,0.571975,0.598750,0.144904
264,0.628333,0.554963,0.902083,0.128125,0,0,1,0,1,0,...,0,1,0,1,0,0,0.628333,0.554963,0.902083,0.128125


In [25]:
y_pred = LR.predict(X_test)

In [26]:
mean_squared_error(y_test, y_pred,squared=False)

774.1964948492608

In [27]:
r2_score(y_test,y_pred)

0.8493628241460262

In [28]:
LR.coef_

array([[-1.22890549e+16,  1.84100000e+03, -6.51938310e+02,
        -1.10939306e+03, -9.48828901e+02,  1.09947277e+02,
         1.17238032e+02,  7.86377452e+02, -9.91556146e+02,
         9.70672074e+02, -7.63143963e+01,  5.96582083e+01,
         4.90967465e+02,  8.27721023e+01,  3.26250494e+02,
         1.46371573e+01, -5.47520814e+02, -2.31204325e+02,
         5.57430099e+02,  1.84265251e+02, -4.26014361e+02,
        -3.40743551e+02,  2.85314650e+02, -2.90658400e+02,
         1.16649238e+02, -3.42685196e+02, -5.20241838e+01,
        -4.69504437e+01,  6.56330329e+01,  6.73055989e+01,
         1.36605409e+02,  2.31689529e+02,  7.52749021e+02,
        -1.04601562e+03,  1.22890549e+16,  1.90237500e+03,
        -6.93437500e+02, -1.15376562e+03],
       [-1.22890549e+16,  1.84100000e+03, -6.51938310e+02,
        -1.10939306e+03, -9.48828901e+02,  1.09947277e+02,
         1.17238032e+02,  7.86377452e+02, -9.91556146e+02,
         9.70672074e+02, -7.63143963e+01,  5.96582083e+01,
         4.90

**Year & Holiday** these are the features which have significant contribution. **The model has a good fit with respect to the dataset.**